## Import Required Libraries

In [1]:
import json
import time
import os
import csv
import json
import numpy as np
import pandas as pd
import random
from itertools import combinations
import math
import sys
import pickle

## Load information about Playlist and Tracks

In [2]:
t0 = time.time()
# Load data matching playlist ID to Track ID
df_play_track = pd.read_csv('./data/track_w_playlist_id.csv')

# Load file containing the Track names
fr = open('./data/tracks_w_names.csv', 'r', encoding="utf-8")
reader = csv.reader(fr)
track_name = []
for row in reader:
    track_name.append(row[1])
fr.close()

In [3]:
# Load file containing candidate pair 
fr = open("./data/candidate_pair_1rows_10bands.pickle", "rb")
can_pairs = pickle.load(fr, encoding='bytes')
fr.close()

# Sort data to make it easier to search
can_pairs = can_pairs[can_pairs[:, 0].argsort()]

## Define Function to Recommend Tracks

In [4]:
def song_recommendation(playlist_track_ids, k, candidate_pairs, track_with_name):
    
    # Get all similar tracks using candidate pairs
    track_ids = np.array([], dtype=int)
    for i in playlist_track_ids:
        idx_start = np.searchsorted(candidate_pairs[:, 0], i, side='left')
        idx_end = np.searchsorted(candidate_pairs[:, 0], i, side='right')
        can_track_ids = candidate_pairs[np.array(range(idx_start,idx_end+1)),1]
        track_ids = np.append(track_ids, can_track_ids)
    
    # Get count of number of each recommended track
    track_rec_id_count = dict([])
    for track in track_ids:
        count = track_rec_id_count.get(track, 0)
        track_rec_id_count[track] = count + 1
    
    # Convert track dictionary to list for easier use
    track_vs_count = []
    for key, value in track_rec_id_count.items():
        track_vs_count.append([value, key])
    
    # Sort track recommendation by suggestion count
    track_vs_count.sort(reverse=True)
    track_vs_count = np.array(track_vs_count)
    
    # Return top K results
    final_ids = track_vs_count[0:k,1]
    track_names = []
    for ele in final_ids:
        track_names.append(track_with_name[ele])
    return track_names
    

## Playlist Addition Recommedation 

In [229]:
# Get list of tracks from a specific playlist
playlist_track_list = df_play_track[df_play_track['Playlist ID'] == 0]["Track ID"].to_list()

# Get number of input and test songs from playlist
num_input_songs = round(len(playlist_track_list)*0.8)
num_test_songs = len(playlist_track_list)-num_input_songs

# Get random set of input songs
random.shuffle(playlist_track_list)
input_songs = []
for i in range(0,num_input_songs):
    input_songs.append(playlist_track_list.pop())
    
# Get random set of test songs
test_songs = [] 
for i in range(0,num_test_songs):
    test_songs.append(playlist_track_list.pop())

# Get song recommendations
recom_songs = song_recommendation(input_songs, num_test_songs, can_pairs, track_name)

In [230]:
# Display recommendation songs
print('Recommendations:')
for i in range(0,num_test_songs):
    print(recom_songs[i])
    
# Display test songs
print('\nActual Songs:')
for i in range(0,num_test_songs):
    print(track_name[test_songs[i]])

Recommendations:
Sk8er Boi - Avril Lavigne
Promiscuous - Nelly Furtado
Eenie Meenie - Justin Bieber
Right Now (Na Na Na) - Akon
A Thousand Miles - Vanessa Carlton
Beautiful Girls - Sean Kingston
Nobody's Perfect - Hannah Montana
Apologize - OneRepublic
Boom Boom Pow - The Black Eyed Peas
Fire Burning - Sean Kingston

Actual Songs:
Run It! - Chris Brown
Livin on Sunday - Cris Cab
Whatcha Say - Jason Derulo
Toxic - Britney Spears
Closer - Ne-Yo
Forever - Chris Brown
Lose Control (feat. Ciara & Fat Man Scoop) - Missy Elliott
Jumpin', Jumpin' - Destiny's Child
Promiscuous - Nelly Furtado
Miss Independent - Ne-Yo
